In [2]:
# process plans for running the humanness experiments 

import os
import openai
from dotenv import load_dotenv
import numpy as np 
import pandas as pd 
import random

load_dotenv()

openai.api_key = "" # TODO: fill in with own key

random.seed(10)

In [3]:
# to start, predetermine condition numbers
# focusing just on single constraints

goal_type = "constrained_single"#"unconstrained"
save_dir = "./gpt3_generations/"

# early indices = maximally conditioned
# later indices = minimally conditioned (in case we don't use this in the future)

max_cond_pth = f"{save_dir}/max_inductive_{goal_type}_plans.csv"
max_df = pd.read_csv(max_cond_pth)

min_cond_pth = f"{save_dir}/min_inductive_{goal_type}_plans.csv"
min_df = pd.read_csv(min_cond_pth)

In [34]:
# shuffle and split each of into subsets of num_in_batch (e.g., 45) plans 

def compute_plan_batches(df, num_in_batch=45): 
    df = df.sample(frac=1).reset_index(drop=True)
    df["batch_idx"] = [idx // num_in_batch for idx in df.index]
    num_tot_batches = len(set(df["batch_idx"]))
    return df, num_tot_batches

# now, compute the exact condition number 
# [NOTE: need to update to handle more goals]
num_in_batch = 45
max_df, num_tot_batches = compute_plan_batches(max_df, num_in_batch)
min_df, _ = compute_plan_batches(min_df, num_in_batch)
# now, increment batch idx to capture earlier plans (from max cond condition)
min_df["batch_idx"] += num_tot_batches
merged_df = pd.concat([max_df,min_df], ignore_index=True)

In [38]:
print(len(merged_df))
merged_df

180


,Unnamed: 0,Unnamed: 0.1,goal,plan,id,prompt,goal_type,batch_idx
0,44,14,"Goal: Build a bookshelf, without using wood.",I would use cardboard and duct tape to build a...,gpt-3,Goal: Get your sofa onto the roof of your hous...,constrained_single,0
1,83,53,Goal: Create a safe landing for a falling skyd...,I would find the highest building in the area....,gpt-3,"Goal: Make a fort underwater, without using an...",constrained_single,0
2,60,30,Goal: Cool down in a record-breaking heat wave...,"I would go to a pool, or jump in a lake.",gpt-3,Goal: Stop your canoe from falling down the wa...,constrained_single,0
3,15,15,Goal: Cool down in a record-breaking heat wave...,I would make a fan out of a bunch of leaves an...,gpt-3,"Goal: Keep the plants in your garden alive, wi...",NaN,0
4,68,38,Goal: Cool down in a record-breaking heat wave...,I would find a way to make a fan. I would make...,gpt-3,"Goal: Clean the dirty dishes, without using so...",constrained_single,0
...,...,...,...,...,...,...,...,...
175,81,51,Goal: Create a safe landing for a falling skyd...,I would try to move the person away from the m...,gpt-3,"Goal: Escape quicksand, without using a branch...",constrained_single,3
176,28,28,Goal: Create a safe landing for a falling skyd...,"I would build a trampoline, but I would make s...",gpt-3,"Goal: Clean the dirty dishes, without using so...",constrained_single,3
177,30,0,"Goal: Build a bookshelf, without using wood.",I would use the same design as the bookshelf I...,gpt-3,"Goal: Make a fort underwater, without using an...",constrained_single,3
178,56,26,Goal: Cool down in a record-breaking heat wave...,"I would go to a pool, and fill up a bucket wit...",gpt-3,Goal: Build a float to dazzle the crowd at the...,constrained_single,3


In [41]:
merged_df.to_csv(f"{save_dir}single_constrained_pilot.csv")